# 3_1 Comparison experiments
This  section is to compare the Group trained model with individual trained models performance (cross validation applied)

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import optim, nn
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices
from utils import *
from model import *



# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Device: ",device,"Device Count: ", torch.cuda.device_count(), "Device Name: ",torch.cuda.get_device_name()  )
    print("Torch version:", torch.__version__)
    
device

Device:  cuda Device Count:  2 Device Name:  Tesla V100-PCIE-16GB
Torch version: 1.7.1


device(type='cuda')

In [52]:
import sys

### imports
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Turn off TensorFlow logging
import tensorflow.compat.v1 as tf # maintain compatibility with TensorFlow 2.2.0

import keras
# from tensorflow.compat.v1.keras import backend as K # changed for compatibility with TensorFlow 2.2.0
import numpy as np
import pandas as pd
import random
#from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
import loadfile
import addons

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from keras.utils import np_utils
from keras.regularizers import l1
from keras.models import load_model, save_model

from tensorflow.compat.v1.keras import backend as K
import numpy as np

shimmer_global_mean = [-0.012359981,-0.0051663737,0.011612018,
                        0.05796114,0.1477952,-0.034395125 ]

shimmer_global_stddev = [0.05756385,0.040893298,0.043825723,
                        17.199743,15.311142,21.229317 ]

shimmer_trended_mean = [-0.000002,-0.000002,-0.000000,
                0.058144,0.147621,-0.033260 ]

shimmer_trended_stddev = [0.037592,0.034135,0.032263,
                17.209038,15.321441,21.242532 ]

all_zero_means = [0,0,0,0,0,0]

meanvals = all_zero_means
stdvals = shimmer_trended_stddev


random_seed  = 1000
batch_size = 64

In [44]:
EPOCHS = 30
winmin = 3 
stridesec = 15

print_settings(winmin,stridesec, EPOCHS)
# Load the dataset
meal_data_train = Person_MealsDataset(person_name= "lawler", file_name = "all_files_list", winmin = winmin,stridesec = stridesec)
# meal_data_train = Person_MealsDataset(person_name= "adam", file_name = "train_files", winmin = winmin,stridesec = stridesec)
# meal_data_test = Person_MealsDataset(person_name= "adam", file_name = "test_files", winmin = winmin,stridesec = stridesec)


*****************************************************************

Execution Started at 12/20/2020, 15:21:17
WindowLength: 3.00 min (2700 datum)	Slide: 15 (225 datum)	Epochs:30

Loading Dataset ...
Loading File:  ../data/IndividualData/lawler-data/10.10/10.10.shm
Loading File:  ../data/IndividualData/lawler-data/10.12/10.12.shm
Loading File:  ../data/IndividualData/lawler-data/10.14/10.14.shm
Loading File:  ../data/IndividualData/lawler-data/10.16/10.16.shm
Loading File:  ../data/IndividualData/lawler-data/10.19/10.19.shm
Loading File:  ../data/IndividualData/lawler-data/10.20/afternoon_2hr33min/10.20.shm
Loading File:  ../data/IndividualData/lawler-data/10.20/evening_2hr_20min/10.20.shm
Loading File:  ../data/IndividualData/lawler-data/10.21/10.21.shm
Loading File:  ../data/IndividualData/lawler-data/10.26/10.26.shm
Loading File:  ../data/IndividualData/lawler-data/10.28/10.28.shm
Loading File:  ../data/IndividualData/lawler-data/10.31/10.31.shm
Loading File:  ../data/IndividualData/l

In [29]:
#random.seed(random_seed)

# load
trainingsamples,traininglabels =  meal_data_train.data_indices, meal_data_train.labels
testsamples,testlabels =  meal_data_test.data_indices, meal_data_test.labels

train_shuffledUnderSampledBalancedIndices = balance_data_indices(traininglabels,mode="under", shuffle=True,random_state = random_seed,replace= False)
test_shuffledUnderSampledBalancedIndices = balance_data_indices(testlabels,mode="under", shuffle=True,random_state = random_seed,replace= False)
print("Train set size: %d, Test set size: %d"%(len(train_shuffledUnderSampledBalancedIndices),len(test_shuffledUnderSampledBalancedIndices)))


NameError: name 'meal_data_test' is not defined

In [45]:
train_indices, valid_indices = split_train_test_indices(X= train_shuffledUnderSampledBalancedIndices,
                                                        y = traininglabels, test_size = 0.2,
                                                       random_seed = random_seed)

Train set size: 8620, with 4310 positive samples and 4310 negative samples
Test set size: 2156, with 1078 positive samples and 1078 negative samples


In [46]:
trainingsamples,traininglabels =  meal_data_train.data_indices, meal_data_train.labels
train_shuffledUnderSampledBalancedIndices = balance_data_indices(traininglabels,mode="under", shuffle=True,random_state = random_seed,replace= False)
train_indices, test_indices = split_train_test_indices(X= train_shuffledUnderSampledBalancedIndices,
                                                        y = traininglabels, test_size = 0.2,
                                                       random_seed = random_seed)

train_indices, valid_indices = split_train_test_indices(X= train_indices,
                                                        y = traininglabels, test_size = 0.2,
                                                       random_seed = random_seed)

Train set size: 2875, with 1437 positive samples and 1438 negative samples
Test set size: 719, with 360 positive samples and 359 negative samples
Train set size: 2300, with 1150 positive samples and 1150 negative samples
Test set size: 575, with 287 positive samples and 288 negative samples


In [47]:
a = set(train_indices)
b = set(valid_indices)
c=set(test_indices)
d = a.intersection(b).intersection(c)
d

set()

In [48]:
train_set_balanced = torch.utils.data.Subset(meal_data_train, train_indices)
valid_set_balanced = torch.utils.data.Subset(meal_data_train, valid_indices)

# test_set_balanced = torch.utils.data.Subset(meal_data_test, test_shuffledUnderSampledBalancedIndices)

train_loader = torch.utils.data.DataLoader(train_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)

# test_loader = torch.utils.data.DataLoader(test_set_balanced ,batch_size=batch_size, shuffle=True,num_workers=2)

print("Data Loader Created")

Data Loader Created


In [49]:
from model import *
from utils import *

torch.manual_seed(random_seed)
input_shape = (train_set_balanced[0][0].shape[0], train_set_balanced[0][0].shape[1])
model_4 = Discriminator_ResNet( ngpu=1, input_shape =input_shape , out_fea = 1)
# Since I use a dynamic created layer in network, need to input a sample to initialize the model first
_ = model_4(torch.rand((1, input_shape[0],input_shape[1])))
model_4.to(device)

criterion = nn.BCEWithLogitsLoss()

optimizer_4 = optim.Adam(model_4.parameters(),lr=0.01,  weight_decay=0.1)
lrscheduler_4 = optim.lr_scheduler.ReduceLROnPlateau(optimizer_4, mode='min',patience= 2, factor = 0.1,threshold=0.01)
dataloader = (train_loader, valid_loader)
model_4, best_model_4,val_score_4,loss_ls_4, train_acc_ls_4, valid_acc_ls_4 = train_model(model_4,dataloader, optimizer_4, 
                                                                    criterion, lrscheduler_4, device= device,
                                                                    n_epochs=30, patience = 5, l1_enabled=False,
                                                                    checkpoint_name ="../models/lawler_models/checkpoint_model_resnet.pt" )

Training set batch amounts: 36
Test set : 9
Start Training..
Epoch: 0,  Epoch_Loss: 1.4536, Train Acc: 52.9130 %, Train Recall: 0.5139 
Validation Acc:  52.8696 %,  Validation Recall: 0.8467 
Checkpoint Saved


Epoch: 1,  Epoch_Loss: 0.6582, Train Acc: 62.6522 %, Train Recall: 0.6443 
Validation Acc:  53.5652 %,  Validation Recall: 0.4042 
Checkpoint Saved


Epoch: 2,  Epoch_Loss: 0.5968, Train Acc: 68.0435 %, Train Recall: 0.6757 
Validation Acc:  54.7826 %,  Validation Recall: 0.2091 
Checkpoint Saved


Epoch: 3,  Epoch_Loss: 0.5674, Train Acc: 69.7391 %, Train Recall: 0.7000 
Validation Acc:  63.6522 %,  Validation Recall: 0.7422 
Checkpoint Saved


Epoch: 4,  Epoch_Loss: 0.5240, Train Acc: 74.9565 %, Train Recall: 0.7304 
Validation Acc:  78.4348 %,  Validation Recall: 0.7596 
Checkpoint Saved


Epoch: 5,  Epoch_Loss: 0.5142, Train Acc: 76.3478 %, Train Recall: 0.7565 
Validation Acc:  76.5217 %,  Validation Recall: 0.6411 


Epoch: 6,  Epoch_Loss: 0.5037, Train Acc: 76.8696 %, Tra

In [50]:
meal_data_test = Person_MealsDataset(person_name= "lawler", file_name = "test_files", winmin = winmin,stridesec = stridesec)
# test_set_balanced = torch.utils.data.Subset(meal_data_test, test_indices)
test_loader = torch.utils.data.DataLoader(meal_data_test ,batch_size=batch_size, shuffle=True,num_workers=2)

best_model_4.eval()
acc, recall = eval_model(best_model_4, test_loader,device)
print("Test Accuracy:", acc)

Loading Dataset ...
Loading File:  ../data/IndividualData/lawler-data/10.10/10.10.shm
Loading File:  ../data/IndividualData/lawler-data/10.12/10.12.shm
Loading File:  ../data/IndividualData/lawler-data/10.14/10.14.shm
Loading File:  ../data/IndividualData/lawler-data/10.16/10.16.shm
Test Accuracy: 77.99166948103118


In [ ]:
def cross_validation(k, model, ):

In [60]:
import sys

### imports
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Turn off TensorFlow logging
import tensorflow.compat.v1 as tf # maintain compatibility with TensorFlow 2.2.0
tf.disable_eager_execution()

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
import keras
# from tensorflow.compat.v1.keras import backend as K # changed for compatibility with TensorFlow 2.2.0
import numpy as np
import pandas as pd
import random
#from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
import loadfile
import addons

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from keras.utils import np_utils
from keras.regularizers import l1
from keras.models import load_model, save_model

from tensorflow.compat.v1.keras import backend as K
import numpy as np

shimmer_global_mean = [-0.012359981,-0.0051663737,0.011612018,
                        0.05796114,0.1477952,-0.034395125 ]

shimmer_global_stddev = [0.05756385,0.040893298,0.043825723,
                        17.199743,15.311142,21.229317 ]

shimmer_trended_mean = [-0.000002,-0.000002,-0.000000,
                0.058144,0.147621,-0.033260 ]

shimmer_trended_stddev = [0.037592,0.034135,0.032263,
                17.209038,15.321441,21.242532 ]

all_zero_means = [0,0,0,0,0,0]

meanvals = all_zero_means
stdvals = shimmer_trended_stddev


random_seed  = 1000
batch_size = 64

In [61]:
balancedData,balancedLabels = meal_data_train.get_subset( train_indices)
valid_balancedData,valid_balancedLabels = meal_data_train.get_subset( valid_indices)

meal_data_test = Person_MealsDataset(person_name= "lawler", file_name = "test_files", winmin = winmin,stridesec = stridesec)
balancedData_test,balancedLabels_test = meal_data_train.get_subset([i for i in range(len(meal_data_test))])

pathtemp = "../models/ActiModels/M_F_"
modelpath = pathtemp + "{:f}Min.h5".format(winmin)
jsonpath = pathtemp + "{:f}Min.json".format(winmin)
winlength = int(winmin * 60 * 15)
step = int(stridesec * 15)
start_time = datetime.now()

mcp_save = keras.callbacks.ModelCheckpoint(modelpath, save_best_only=True, monitor='accuracy')
model = Sequential()
model.add(Conv1D(10, 44, strides=2,activation='relu', input_shape=(winlength, 6)))
model.add(Conv1D(10, 20, strides=2, activation='relu',kernel_regularizer=keras.regularizers.l1(0.01)))
model.add(Conv1D(10, 4, strides=2, activation='relu',kernel_regularizer=keras.regularizers.l1(0.01)))
model.add(GlobalAveragePooling1D())
model.add(Dense(200, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])

K.get_session().close()
K.set_session(tf.Session())
K.get_session().run(tf.global_variables_initializer())
H = model.fit(x=balancedData, y = balancedLabels,
                       validation_data=[valid_balancedData, valid_balancedLabels],
                    epochs = EPOCHS, batch_size=256, verbose=1, validation_split=0.2,
                    callbacks=[mcp_save]) # removed addons.LossHistory(jsonpath) for compatibility with TensorFlow 2.2.0, needs to be re-added at some point

print("Max value: ", max(H.history['accuracy']), " at epoch", np.argmax(H.history['accuracy']) + 1)

from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
predictions = model.predict(x=balancedData_test)
threshold = 0.5
acc =  accuracy_score(predictions>threshold,balancedLabels_test)
recall = recall_score(predictions>threshold,balancedLabels_test)
auc = roc_auc_score(predictions>threshold,balancedLabels_test)
print("Test Accuracy:", acc)
print("Recall Accuracy:", recall)
print("AUC Score:", auc)


Loading Dataset ...
Loading File:  ../data/IndividualData/lawler-data/10.10/10.10.shm
Loading File:  ../data/IndividualData/lawler-data/10.12/10.12.shm
Loading File:  ../data/IndividualData/lawler-data/10.14/10.14.shm
Loading File:  ../data/IndividualData/lawler-data/10.16/10.16.shm
Train on 2300 samples, validate on 575 samples
Epoch 1/30
2300/2300 [==============================] - ETA: 0s - loss: 2.3800 - accuracy: 0.5283WARNING:tensorflow:From /home/wenkanw/.conda/envs/mlenv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training_v1.py:2048: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
2300/2300 [==============================] - 1s 620us/sample - loss: 2.3800 - accuracy: 0.5283 - val_loss: 2.2612 - val_accuracy: 0.6365
Epoch 2/30
2300/2300 [==============================] - 1s 6